In [118]:
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain.vectorstores.chroma import Chroma

In [123]:
from core import get_embedding_function
from docs_pipeline import load_document_from_pdf, split_documents
from vector_db import add_to_chroma
from rag_pipeline import query_rag

In [ ]:
DATA_PATH = '../data'

EMBEDDING_MODEL_NAME = 'nomic-embed-text'
CHAT_MODEL_NAME = 'phi3'

In [ ]:
def prepare_db()-> None:
    documents = load_document_from_pdf(DATA_PATH)
    chunks = split_documents(documents)
    add_to_chroma(chunks, get_embedding_function(EMBEDDING_MODEL_NAME))
    

In [ ]:
def read_prompt_template(template_path: str) -> str:
    with open(template_path, 'r') as f:
        return f.read()

In [ ]:
prepare_db()

In [ ]:
db = Chroma(
    persist_directory=CHROMA_PATH,
    embedding_function=get_embedding_function(EMBEDDING_MODEL_NAME),
)

In [124]:
query = ('In a direct time-of-flight (ToF) laser rangefinder or scanner the results of distance measurements'
         'a) greatly depend on the optical properties of the observed surface, particularly the roughness of the surface'
         'b) little depend on the optical properties of the observed surface.'
         'c) greatly depend on the optical properties of the observed surface, particularly the absorbtion coefficient.')
         
prompt_template = read_prompt_template('prompt.tplt')
response, prompt, sources = query_rag(query, prompt_template, Ollama(model=CHAT_MODEL_NAME), db)
response

"Response: In a direct time-of-flight (ToF) laser rangefinder or scanner, the results of distance measurements little depend on the optical properties of the observed surfaceb) little depend on the optical properties of the observed surface.\n\n---\nNote: The provided answer is based sole Written content from given context and does not consider other potential information sources.\nSources: ['data\\\\aimp-3.pdf:12:0', 'data\\\\aimp-3.pdf:4:0', 'data\\\\aimp-3.pdf:14:0', 'data\\\\aimp-3.pdf:7:1', 'data\\\\aimp-3.pdf:13:0']"